<a href="https://colab.research.google.com/github/SungjooHwang/SungjooHwang/blob/main/Ex09_Practice_3_Human_Action_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code is based on: https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/ **

**The Dataset**

For the Practice #3 in the Lecture Note #09, Use 'finaldata.csv'that include acceleration signal features and labels of actions ('1' for spreading mortar, '2' for laying block, '3' for adjusting, and '4'for removing mortar)
You can also use your own dataset.



**Importing Libraries**

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

**Importing the Dataset**

Import the dataset and load it into our pandas dataframe,

In [55]:
dataset = pd.read_csv("finaldata.csv")

In [56]:
dataset.head()

,MeanX,MeanY,MeanZ,SkewX,SkewY,SkewZ,MaxX,MaxY,MaxZ,MinX,MinY,MinZ,RangeX,RangeY,RangeZ,stdX,stdY,stdZ,KurtosisX,KurtosisY,KurtosisZ,CorrXY,CorrYZ,CorrZX,EnergyX,EnergyY,EnergyZ,EntropyX,EntropyY,EntropyZ,action
0,-22.602273,55.329545,-5.079545,-0.430630,-0.152069,0.716635,78,127,105,-128,-58,-85,206,185,190,42.338684,43.120980,28.553729,3.736030,2.443957,5.255820,-0.006740,0.185144,0.117068,77553.723366,80642.530185,35446.530185,10.118275,3.188343,19.139626,1
1,-18.102273,58.488636,-5.340909,-0.573902,-0.193898,1.209931,75,127,105,-128,-58,-54,203,185,159,40.263511,37.033379,27.008252,4.070303,3.187264,5.999251,-0.068927,0.267239,0.278444,70430.973366,59653.646662,31609.886364,12.220491,2.672048,18.039167,1
2,-19.681818,58.375000,-11.954545,-0.664967,0.602729,0.859530,48,127,86,-127,-6,-83,175,133,169,31.549467,31.032172,28.925949,5.124737,2.837830,4.734259,0.066632,0.359750,0.262228,43265.482955,41845.589844,36148.846591,8.392115,2.077043,11.524579,1
3,-15.625000,55.693182,-10.590909,-0.328796,0.430409,0.291269,68,127,83,-128,-21,-83,196,148,166,34.439456,29.261547,27.805828,4.469902,3.875835,4.190040,-0.004686,0.201368,0.271803,51586.402344,37230.854048,33629.995739,11.412033,2.188670,13.520706,1
4,-20.125000,52.295455,-16.409091,0.238599,0.372197,-0.180156,85,127,55,-128,-21,-91,213,148,146,36.542469,27.227082,24.544417,3.668240,3.833551,3.702255,-0.045861,0.286993,0.184446,58083.558594,32247.096591,25912.370739,9.665516,2.045628,7.365125,1


**Preprocessing**

The next step is to split our dataset into its attributes and labels. 

The X variable contains the first 30 columns of the dataset (i.e. signal features) while y contains the labels of actions.

In [57]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 30].values
# print(y)

**Feature Scaling**

Before making any actual predictions, it is always a good practice to scale the features so that all of them can be uniformly evaluated. Wikipedia explains the reasoning pretty well:

*Since the range of values of raw data varies widely, in some machine learning algorithms, objective functions will not work properly without normalization. For example, the majority of classifiers calculate the distance between two points by the Euclidean distance. If one of the features has a broad range of values, the distance will be governed by this particular feature. Therefore, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance.*

The gradient descent algorithm (which is used in neural network training and other machine learning algorithms) also converges faster with normalized features.



In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

**Training and Cross Validation**

The first step in the training and cross validation phase is simple. 

In [59]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=300, random_state=0)


Next, to implement cross validation, the *cross_val_score* method of the *sklearn.model_selection* library can be used. The *cross_val_score* returns the accuracy for all the folds. The first parameter is estimator which basically specifies the algorithm that you want to use for cross validation. The second and third parameters, X and y, i.e. features and labels. Finally the number of folds is passed to the cv parameter as shown in the following code:

In [60]:
from sklearn.model_selection import cross_val_score
all_accuracies = cross_val_score(estimator=classifier, X=X, y=y, cv=5)

In [61]:
print(all_accuracies)
print(all_accuracies.mean())
print(all_accuracies.std())

[0.91935484 1.         0.9516129  0.8852459  0.8852459 ]
0.9282919090428345
0.04352403430590002


**(Optional) Grid Search for Parameter Selection**

A machine learning model has two types of parameters. The first type of parameters are the parameters that are learned through a machine learning model while the second type of parameters are the hyper parameter that we pass to the machine learning model.

In the last section, we used the Random Forest algorithm. The number of estimators we used for the algorithm was 300. Similarly in KNN algorithm we have to specify the value of K and for SVM algorithm we have to specify the type of Kernel. These estimators - the K value and Kernel - are all types of hyper parameters.

Normally we randomly set the value for these hyper parameters and see what parameters result in best performance. However randomly selecting the parameters for the algorithm can be exhaustive.

Also, it is not easy to compare performance of different algorithms by randomly setting the hyper parameters because one algorithm may perform better than the other with different set of parameters. And if the parameters are changed, the algorithm may perform worse than the other algorithms.

Therefore, instead of randomly selecting the values of the parameters, a better approach would be to develop an algorithm which automatically finds the best parameters for a particular model. Grid Search is one such algorithm.

To implement the Grid Search algorithm we need to import GridSearchCV class from the sklearn.model_selection library.
The first step you need to perform is to create a dictionary of all the parameters and their corresponding set of values that you want to test for best performance. The name of the dictionary items corresponds to the parameter name and the value corresponds to the list of values for the parameter.
Let's create a dictionary of parameters and their corresponding values for our Random Forest algorithm. Details of all the parameters for the random forest algorithm are available in the Scikit-Learn docs.

In [62]:
from sklearn.model_selection import GridSearchCV
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

Take a careful look at the above code. Here we create grid_param dictionary with three parameters n_estimators, criterion, and bootstrap. The parameter values that we want to try out are passed in the list. For instance, in the above script we want to find which value (out of 100, 300, 500, 800, and 1000) provides the highest accuracy.

Similarly, we want to find which value results in the highest performance for the criterion parameter: "gini" or "entropy"? The Grid Search algorithm basically tries all possible combinations of parameter values and returns the combination with the highest accuracy. For instance, in the above case the algorithm will check 20 combinations (5 x 2 x 2 = 20).

The Grid Search algorithm can be very slow, owing to the potentially huge number of combinations to test. Furthermore, cross validation further increases the execution time and complexity.

Once the parameter dictionary is created, the next step is to create an instance of the GridSearchCV class. You need to pass values for the estimator parameter, which basically is the algorithm that you want to execute. The param_grid parameter takes the parameter dictionary that we just created as parameter, the scoring parameter takes the performance metrics, the cv parameter corresponds to number of folds, which is 5 in our case, and finally the n_jobs parameter refers to the number of CPU's that you want to use for execution. A value of -1 for n_jobs parameter means that use all available computing power. This can be handy if you have large number amount of data.

In [63]:
gd_sr = GridSearchCV(estimator=classifier,
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)



Once the GridSearchCV class is initialized, the last step is to call the fit method of the class and pass it the training  set



In [66]:
gd_sr.fit(X, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=300, n_jobs=None,
                                              oob_score=False, random_state=0,
                                   

Once the method completes execution, the next step is to check the parameters that return the highest accuracy. To do so, print the sr.best_params_ attribute of the GridSearchCV object.

In [67]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 100}


The last and final step of Grid Search algorithm is to find the accuracy obtained using the best parameters. 

In [69]:
best_result = gd_sr.best_score_
print(best_result)

0.9479111581173981
